In [ ]:
# Notebook to create subsampled (e.g. 75%, 50%, 25%, 10%, 5%, 1%, 0.1%) of training data to evaluate
# training in the low-data setting

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
join = os.path.join
from tqdm import tqdm
from skimage import transform 
from torch.utils.data import Dataset, DataLoader
import torch.multiprocessing as mp
import sys
sys.path.append('./modified_medsam_repo')
import random
from datetime import datetime
import pandas as pd
import nibabel as nib
import pickle

In [2]:
full_df = pd.read_csv('/gpfs/data/luilab/karthik/pediatric_seg_proj/path_df_constant_bbox.csv')
splits = pd.read_pickle('/gpfs/data/luilab/karthik/pediatric_seg_proj/train_val_test_split.pickle')

In [17]:
percents_interested = [100, 75, 50, 25, 10, 5, 2.5, 1, 0.5, 0.1]

src_path = '/gpfs/data/luilab/karthik/pediatric_seg_proj/subset_train_id_dfs_pooled'
np.random.seed(2024)

last_train_list = splits['train']
for xo in percents_interested:
    mult = xo / 100.0
    num_train_to_sample = round(len(splits['train']) * mult)
    
    train_idxs = np.random.choice(last_train_list, size = num_train_to_sample, replace=False)
    new_dict = {'train': train_idxs, 'val': splits['val'], 'test':splits['test']}
    

    pd.to_pickle(new_dict, f'{src_path}/{xo}.pkl')
    #print(train_idxs)
    last_train_list = train_idxs